# Drosix motor identification

## Process

1. Put the drone on the test rig
2. Place a cooking scale below one side
3. Place a plank between the scale and the two arms so that the two arms push equaly on the scale (keep the drone as horizontal as possible)
4. Reset the scale to supress the resting weight from the frame on the scale
5. Activate the two motors on the opposite side of the scale
6. After few second (stable weight) note the measure

## Measures

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

G = 9.8  # Earth's gravity
throttle = np.array([0, 10, 20, 30, 35, 40, 45, 50, 55, 60, 65, 70])
weight = np.array([0, 71, 186, 300, 370, 425, 490, 540, 600, 650, 720, 800]) * 1e-3 * G

plt.plot(throttle, weight)
plt.xlabel("Throttle (%)")
plt.ylabel("Thrust (N)")
plt.grid()



## Expected result and shape

The drone model gives the following mapping between the throttle $\sigma$ and the thrust $T$:

$$
T = (\omega_b + C_r * \sigma)^2 * C_t
$$

Where $\omega_b$ and $C_r$ are the linear function parameters modeling the ESC behavior and $C_t$ is the propeller thrust coefficient

In [ ]:
t_shape = np.linspace(0, 100, 10)
w_shape = np.square(50 + 500 * t_shape) * 1e-5

plt.plot(t_shape, w_shape)
plt.xlabel("Throttle (%)")
plt.ylabel("Thrust")
plt.yticks([]) 

## Identification

In [ ]:
from scipy.optimize import curve_fit


def model(x, wb, cr, ct):
    return np.square(wb + cr * x / 100) * ct * 2

popt, pcov = curve_fit(model, throttle[2:], weight[2:],
                       bounds=([10, 500, 9e-6], [200, 800, 2e-5]))
print(popt)

plt.plot(throttle, weight, 'k', label="Measure")
plt.plot(t_shape, model(t_shape, *popt), '--b', label="Model")
plt.plot(t_shape, model(t_shape, 57, 673, 1.2e-5), '--r', label="Drosix")
plt.xlabel("Throttle (%)")
plt.ylabel("Thrust (N)")
plt.legend()